<a href="https://colab.research.google.com/github/elizabethts/DS5-kaggle-scalar-coupling/blob/master/submissions/First_Submission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# !pip install kaggle
# !mkdir .kaggle

import json
token = ''
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)
    
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
!kaggle config set -n path -v{/content}

!chmod 600 /root/.kaggle/kaggle.json


!kaggle competitions download -c champs-scalar-coupling

- path is now set to: {/content}
 99% 44.0M/44.5M [00:01<00:00, 20.4MB/s]
100% 44.5M/44.5M [00:01<00:00, 34.1MB/s]
 37% 5.00M/13.4M [00:00<00:00, 10.2MB/s]
100% 13.4M/13.4M [00:00<00:00, 22.8MB/s]
 78% 49.0M/62.9M [00:01<00:00, 24.5MB/s]
100% 62.9M/62.9M [00:01<00:00, 48.2MB/s]
 94% 46.0M/48.7M [00:01<00:00, 20.2MB/s]
100% 48.7M/48.7M [00:01<00:00, 32.5MB/s]
 84% 73.0M/86.6M [00:01<00:00, 35.0MB/s]
100% 86.6M/86.6M [00:01<00:00, 47.3MB/s]
 94% 5.00M/5.31M [00:00<00:00, 12.0MB/s]
100% 5.31M/5.31M [00:00<00:00, 12.0MB/s]
  0% 0.00/497k [00:00<?, ?B/s]
100% 497k/497k [00:00<00:00, 153MB/s]
 54% 5.00M/9.24M [00:00<00:00, 13.7MB/s]
100% 9.24M/9.24M [00:00<00:00, 23.4MB/s]
 82% 41.0M/49.8M [00:02<00:00, 12.6MB/s]
100% 49.8M/49.8M [00:02<00:00, 23.0MB/s]
  0% 0.00/965k [00:00<?, ?B/s]
100% 965k/965k [00:00<00:00, 66.1MB/s]


In [6]:
from google.colab import files
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
!pip install category_encoders
from category_encoders import OrdinalEncoder
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error
import numpy as np
import xgboost as xgb
from xgboost import XGBRegressor

     |████████████████████████████████| 92kB 1.8MB/s 


In [7]:
!unzip /content/{/content}/competitions/champs-scalar-coupling/dipole_moments.csv.zip
!unzip /content/{/content}/competitions/champs-scalar-coupling/structures.csv.zip
!unzip /content/{/content}/competitions/champs-scalar-coupling/scalar_coupling_contributions.csv.zip

Archive:  /content/{/content}/competitions/champs-scalar-coupling/dipole_moments.csv.zip
  inflating: dipole_moments.csv      
Archive:  /content/{/content}/competitions/champs-scalar-coupling/structures.csv.zip
  inflating: structures.csv          
Archive:  /content/{/content}/competitions/champs-scalar-coupling/scalar_coupling_contributions.csv.zip
  inflating: scalar_coupling_contributions.csv  


In [0]:
trainval = pd.read_csv('/content/{/content}/competitions/champs-scalar-coupling/train.csv.zip')
test = pd.read_csv('/content/{/content}/competitions/champs-scalar-coupling/test.csv.zip')
structures = pd.read_csv('/content/structures.csv')
dipole = pd.read_csv('/content/dipole_moments.csv')
contrib = pd.read_csv('/content/scalar_coupling_contributions.csv')

In [0]:
def merge_train_structures(train, structures):
  
  """This function is used to merge the structures dataset to the 
     original train dataset
     
     Parameters:
     ===========
	   train: train dataframe
	   structures: structures dataframe.
     
     Output:
	   ===========
	   dataframe: merged dataframe
     """
  
  structures = structures.rename({'atom_index': 'atom_index_0',
                                  'x':'x_0', 'y':'y_0', 'z':'z_0',
                                  'atom':'atom_0'}, axis=1)
  
  merged = pd.merge(train, structures, on=['molecule_name', 'atom_index_0'])
  
  structures = structures.rename({'atom_index_0': 'atom_index_1',
                                  'x_0':'x_1', 'y_0':'y_1', 'z_0':'z_1',
                                  'atom_0':'atom_1'}, axis=1)
  
  merged_1 = pd.merge(merged, structures, on=['molecule_name', 'atom_index_1'])
  
  structures = structures.rename({'atom_index_1': 'atom_index',
                                  'x_1':'x', 'y_1':'y', 'z_1':'z',
                                  'atom_1':'atom'}, axis=1)
  
  assert train.shape[0] == merged.shape[0]
  
  return merged_1

In [0]:
merged = merge_train_structures(trainval, structures)

In [0]:
def feature_eng(df):
#   df['dihedral_angle'] = abs(df['x_0'] * df['x_1'] + df['y_0'] * df['y_1'] + df['z_0'] * df['z_1']) / (np.sqrt(np.square(df['x_0']) + np.square(df['y_0']) + np.square(df['z_0'])) * (np.sqrt(np.square(df['x_1']) + np.square(df['y_1']) + np.square(df['z_1']))))
  df['bond'] = df['type'].str[2:]
  df['j_type'] = df['type'].str[:2]
  df['distance'] = np.power((np.power((df['x_1'] - df['x_0']), 2) + np.power((df['y_1'] - df['y_0']), 2) + np.power((df['z_1'] - df['z_0']), 2)), 0.5)

In [0]:
feature_eng(merged)


In [49]:
train, val = train_test_split(merged, random_state=47)
train.shape[0] + val.shape[0] == trainval.shape[0]

True

In [0]:
groups = val['type']

def group_lmae(y_true, y_pred, groups, floor=1e-9):
  maes = (y_true - y_pred).abs().groupby(groups).mean()
  return np.log(maes.map(lambda x: max(x, floor))).mean()

In [52]:
%%time
features = ['type', 'atom_1', 'bond', 'j_type', 'distance']
target = 'scalar_coupling_constant'

X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]

y_train_log = np.log1p(y_train)

xgbr = XGBRegressor(n_estimators=100,
                    random_state=47,
                    n_jobs=-1)

encoder = OrdinalEncoder()

pipe = make_pipeline(encoder, xgbr)
pipe.fit(X_train, y_train)

y_pred = pipe.predict(X_val)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in log1p
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in log1p
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[06:23:07] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
CPU times: user 3min 44s, sys: 565 ms, total: 3min 44s
Wall time: 2min 20s


In [53]:
group_lmae(y_val, y_pred, groups)

0.8295697935626145

In [0]:
test_merged = merge_train_structures(test, structures)

In [0]:
feature_eng(test_merged)

In [0]:
y_pred = pipe.predict(test_merged[features])

In [0]:
sub = pd.DataFrame(data = {
    'id': test_merged['id'],
    'scalar_coupling_constant': y_pred 
})
sub.to_csv('submissionscalar.csv', index=False)

In [60]:
files.download('submissionscalar.csv')

----------------------------------------
Exception happened during processing of request from ('::ffff:127.0.0.1', 34130, 0, 0)
Traceback (most recent call last):
  File "/usr/lib/python3.6/socketserver.py", line 320, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 351, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 364, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.6/socketserver.py", line 724, in __init__
    self.handle()
  File "/usr/lib/python3.6/http/server.py", line 418, in handle
    self.handle_one_request()
  File "/usr/lib/python3.6/http/server.py", line 406, in handle_one_request
    method()
  File "/usr/lib/python3.6/http/server.py", line 639, in do_GET
    self.copyfile(f, self.wfile)
  File "/usr/lib/python3.6/http/server.py", line 800, in copyfile
    shutil.copyfil